# DNBLab Jupyter Notebook Tutorial



### 1. Einrichten der Arbeitsumgebung  <a class="anchor" id="Teil1"></a>

In [1]:
import requests
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata

#URL der SRU-Schnittstelle der DNB: 
base_url = "https://services.dnb.de/sru"

#Anfrage - wir speichern das Ergebnis in die Variable "basic_request":
basic_request = requests.get(base_url)

In [6]:
number = response.find('numberofrecords')
print(number.text, 'Ergebnisse')
#print(number)


27 Ergebnisse


Da die einzelnen Treffer bzw. Werke jeweils durch "record"-Tags gekennzeichnet sind, suchen wir nun nach diesen und speichern sie in der Variable "records" zwischen. Zum Vergleich lassen wir uns im Anschluss noch die Länge der Variable ausgeben und sehen, dass diese mit der Angabe unter "numberofrecords" übereinstimmt. 

ACHTUNG: Dies funkioniert nur bis zu einer Treffermenge von insgesamt 100! Auch bei größeren Treffermengen wird im folgenden maximal die Länge 100 angezeigt - wie wir damit umgehen, folgt weiter unten. 

In [7]:
records = response.find_all('record')
print(len(records), 'Ergebnisse')

27 Ergebnisse


Die Ergebnisse werden als Liste gespeichert, was bedeutet, dass unsere Variable "records" eine Listenvariable ist. Die Ergebnisse stehen dabei jeweils an einem eigenen Listenplatz - bei 9 Ergebnissen gibt es in unserer Liste also 9 Einträge. Da der erste Eintrag allerdings an Listenplatz 0 steht, sind die Plätze von 0-8 mit den 9 Einträgen belegt - dies ist wichtig, wenn wir einzelene Listenplätze adressieren wollen. 

Wir können uns so bspw. den 3. Einträg anzeigen lassen, in dem wir uns den Platz mit der Nummer 2 aufrufen: 

In [61]:
print(records[5])

<record><recordschema>oai_dc</recordschema><recordpacking>xml</recordpacking><recorddata><dc xmlns="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dnb="http://d-nb.de/standards/dnbterms" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
<dc:title>Informationssystem KLARA 1.0 : Klimawandel - Auswirkungen, Risiken, Anpassung ; Analysen spezifischer Verwundbarkeiten und Handlungsoptionen im Land Baden-Württemberg / LfU, Landesanstalt für Umweltschutz Baden-Württemberg ...  Projektleiter: Manfred Stock ...</dc:title>
<dc:creator>Stock, Manfred [Mitwirkender]</dc:creator>
<dc:publisher>Karlsruhe : LfU</dc:publisher>
<dc:date>2005</dc:date>
<dc:identifier xsi:type="dnb:IDN">97609228X</dc:identifier>
<dc:subject>330 Wirtschaft</dc:subject>
<dc:subject>360 Soziale Probleme, Sozialdienste, Versicherungen</dc:subject>
<dc:subject>550 Geowissenschaften</dc:subject>
<dc:format>1 CD-ROM</dc:format>
</dc></recorddata><recordposition>6</recordposit

Um das Ganze nun etwas bequemer nutzen zu können, führen wir die verschiedenen einzelnen Schritte nun in einer Funktion zusammen, die wir dann nur noch mit unserer Wunschabfrage aufrufen müssen, um unsere Ergebnisse zu erhalten: 

In [94]:
def sru_dnb(query): 

    dnb_url = "https://services.dnb.de/sru/dnb"
    parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : query,
                 'recordSchema' : 'oai_dc', 'maximumRecords': '100'} 

    r = requests.get(dnb_url, params = parameter)
    response = soup(r.content)
    records = response.find_all('record')  
    
    return records


In [97]:
myquery = sru_dnb('tit=Klimawandel and jhr=2005')
print(len(myquery), "Ergebnisse")

27 Ergebnisse


Da wir allerdings im Vorfeld meist nicht wissen, wieviele Ergebnisse unsere Suchanfrage erzeugen wird und wir damit rechnen, dass häufig Ergebnismengen von über 100 Treffern vorkommen, passen wir die Funktion nun noch etwas an. Dazu bauen wir eine kleine Schleife, die zunächst abfragt, wieviele Treffer gefunden wurden und dann entscheidet: Werden bis zu 100 Treffer gemeldet gibt es keinen Änderungsbedarf und die Funktion kann so einfach durchlaufen. Werden jedoch mehr als 100 Treffer gefunden, wird die Anfrage in 100er Schritte aufgeteilt und die Ergebnisse jeweils zwischengespeichert. Sobald alle Teile der Anfrage verarbeitet wurden, wird dann das gesammelte Ergebnis übergeben. Die Funktion sieht dann folgendermaßen aus: 

In [99]:
def sru_dnb(query): 

    dnb_url = "https://services.dnb.de/sru/dnb"
    parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : query,
                 'recordSchema' : 'oai_dc', 'maximumRecords': '100'} 

    r = requests.get(dnb_url, params = parameter)
    response = soup(r.content)
    records = response.find_all('record')  
    
    
    if len(records) < 100:
        return records
    
    else: 
        num_results = 100
        i = 101
        
        while num_results == 100:
            
            parameter.update({'startRecord': i})
            r = requests.get(dnb_url, params=parameter)
            xml = soup(r.content)
            new_records = xml.find_all('record')
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

Diese Funktion wird auch bereits im Tutorial ["Wie können Daten mittels der SRU-Schnittstelle abgerufen werden?"](https://www.dnb.de/DE/Professionell/Services/WissenschaftundForschung/DNBLab/dnblab_node.html#doc731014bodyText4) verwendet. 

Eine Abfrage für das Titelstichwort "Klimawandel", kombiniert nun mit den Jahr 2019, ergibt nun 272 Ergebnisse: 

In [101]:
myquery = sru_dnb('tit=Klimawandel and jhr=2019')
print(len(myquery), "Ergebnisse")

273 Ergebnisse


### 4. Verarbeiten der Ergebnisse  <a class="anchor" id="Teil4"></a>

Um mit diesen Ergebnissen nun weiterarbeiten zu können, überführen wir diese aus dem XML in eine tabellarische Form. 

Auch hier arbeiten wir wieder mit einer Funktion, um diese für verschiedene Anfragen nachnutzen zu können. Wir nennen diese Funktion "parse_record", weil wir ihr beim späteren Aufruf die einzelnen Records, die wir durch unsere Anfrage an die SRU-Schnittstelle übergeben werden. Die Funktion soll dann nach einem bestimmten Feld suchen (bspw. Creator, Titel etc.) und uns den Inhalt dieses Feldes zurückgeben. 

Hierfür geben wir den Namespace unseres XML an - dieser ist in der XML-Antwort der Schnittstelle zu finden. Außerdem parsen wir unsere Antwort mit Hilfe der Bibliothek ElementTree (hier kurz ET) in die passende Form zur Weiterverarbeitung. Danach definieren wir, wo der Titel zu finden ist (im Feld beginnend mit "dc:title") und holen uns von dort den Inhalt (den Titeltext). Wenn die Funktion dort nichts findet, gibt sie das des Titeltextes den Text "unknown" zurück:   

In [107]:
def parse_record(record):
    
    ns = {"dc": "http://purl.org/dc/elements/1.1/"}
    xml = ET.fromstring(unicodedata.normalize("NFC", str(record)))
    
    
    #titel
    titel = xml.xpath('.//dc:title', namespaces=ns)
    try:
        titel = titel[0].text
    except:
        titel = "unkown"
        
        
    meta_dict = {"titel":titel}
    
    return meta_dict

Im Anschluss führen wir nun die Funktion aus, in dem wir ihr die einzelnen records aus unserer Antwort "myquery" übergeben. Das Ergebnis ("output") überführen wir dann in ein Dataframe ("df") und lassen uns dieses anzeigen:

In [114]:
output = [parse_record(record) for record in myquery]
df = pd.DataFrame(output)
df

,titel
0,12. Mitteldeutscher Rinder-Workshop in Bernbur...
1,"25. Welt-Klimakonferenz in Madrid, deutschland..."
2,"55 knackige Punkte, wie wir gemeinsam den Klim..."
3,"55 knackige Punkte, wie wir gemeinsam den Klim..."
4,Adaptive potential of the Arctic diatom Thalas...
...,...
268,Der Klimawandel wird weitergehen — eine unbequ...
269,"Fortbildungsangebote zu Klimawandel, Hitze und..."
270,"Globalisierung, Klimawandel, Migration / von red"
271,Nachfrageprognose und Wasserverbrauchssteuerun...


Dieses können wir uns nun als CSV-Datei oder auch direkt im Excel-Format xlsx ausgeben lassen: 

In [115]:
df.to_csv("Titelabfrage.csv", index=False) 
#df.to_excel("Titelabfrage.xlsx", encoding='utf8')

Da wir allerdings nicht nur den Titel in unsere Tabelle bzw. Dataframe überführen wollen, sondern weitere Angaben wie IDN, Creator, Sachschlagworte, Publikationsdatum etc. erweitern wir die Funktion entsprechend. Das Prinzip bleibt dabei gleich. Da es jedoch Felder in Dublin Core gibt, die sich wiederholen können, wurde im Folgenden der Code für die Schlagworte (subject) so angepasst, dass für jeden Titel bis zu drei Schlagworte aus den Daten extrahiert werden können:  

In [121]:
def parse_record(record):
    
    ns = {"dc": "http://purl.org/dc/elements/1.1/", 
          "xsi": "http://www.w3.org/2001/XMLSchema-instance"}
    xml = ET.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath(".//dc:identifier[@xsi:type='dnb:IDN']", namespaces=ns) 
    if idn: 
        idn = idn[0].text
    else:
        idn = 'fail'
    
    
    #titel
    titel = xml.xpath('.//dc:title', namespaces=ns)
    try:
        titel = titel[0].text
    except:
        titel = "unkown"
        
    
    #creator
    creator = xml.xpath('.//dc:creator', namespaces=ns)
    try:
        creator = creator[0].text
    except:
        creator = "unkown"
        
    
    #date
    date = xml.xpath('.//dc:date', namespaces=ns)
    try:
        date = date[0].text
    except:
        date = "unkown"
        
        
        
    #subjects: 
    subject = xml.xpath(".//dc:subject", namespaces=ns) 
    
    if len(subject) == 1: 
        subject1 = subject[0].text
        subject2 = 'N/A'
        subject3 = 'N/A'
    elif len(subject) == 2:
        subject1 = subject[0].text
        subject2 = subject[1].text
        subject3 = 'N/A'
    elif len(subject) == 3:
        subject1 = subject[0].text
        subject2 = subject[1].text
        subject3 = subject[2].text
    else:
        subject1 = 'N/A'
        subject2 = 'N/A'
        subject3 = 'N/A'
        
        
    #weitere IDs: 
    urn = xml.xpath(".//dc:identifier[@xsi:type='tel:URN']", namespaces=ns) 
    if urn: 
        urn = urn[0].text
    else:
        urn = 'N/A'
        
        
    meta_dict = {"idn":idn, "titel":titel, "creator":creator, "date":date, 
                 "subject1":subject1, "subject2":subject2, "subject3":subject3, "urn":urn}
    
    return meta_dict

In [118]:
output = [parse_record(record) for record in myquery]
df = pd.DataFrame(output)
df

,idn,titel,creator,date,subject1,subject2,subject3,urn
0,1206546913,12. Mitteldeutscher Rinder-Workshop in Bernbur...,"Scholz, Heiko [Herausgeber]",2019,"630 Landwirtschaft, Veterinärmedizin",N/A,N/A,N/A
1,1206340592,"25. Welt-Klimakonferenz in Madrid, deutschland...",unkown,2019,ERROR,ERROR,ERROR,N/A
2,1176214233,"55 knackige Punkte, wie wir gemeinsam den Klim...","Matthée, Jörg",2019,"300 Sozialwissenschaften, Soziologie, Anthropo...",N/A,N/A,N/A
3,117864622X,"55 knackige Punkte, wie wir gemeinsam den Klim...","Matthée, Jörg [Verfasser]",2019,330 Wirtschaft,N/A,N/A,urn:nbn:de:101:1-2019022020104008152243
4,1186248742,Adaptive potential of the Arctic diatom Thalas...,"Wolf, Klara Katharina Estrella [Verfasser]",2019,"570 Biowissenschaften, Biologie",N/A,N/A,urn:nbn:de:gbv:46-00107155-14
...,...,...,...,...,...,...,...,...
268,1212955544,Der Klimawandel wird weitergehen — eine unbequ...,"Neubäumer, Renate [Verfasser]",2019,330 Wirtschaft,N/A,N/A,urn:nbn:de:101:1-2020070112021412765574
269,118985712X,"Fortbildungsangebote zu Klimawandel, Hitze und...","Schoierer, Julia [Verfasser]",2019,"610 Medizin, Gesundheit",N/A,N/A,urn:nbn:de:101:1-2019070412392166110088
270,1184762767,"Globalisierung, Klimawandel, Migration / von red",red [Verfasser],2019,"610 Medizin, Gesundheit",N/A,N/A,urn:nbn:de:101:1-2019042819000522717073
271,1177804212,Nachfrageprognose und Wasserverbrauchssteuerun...,"Yildiz, Özgür [Verfasser]",2019,330 Wirtschaft,N/A,N/A,urn:nbn:de:101:1-2019021204072972329687


Damit ist unsere Abfrage deutlich erweitert und wir können sie wieder als CSV exportieren oder auch direkt in Python mit dem Dataframe weiterarbeiten.

In [120]:
df.to_csv("Titelabfrage_erweitert.csv", index=False, encoding='utf8') 

In [122]:
df.to_excel("Titelabfrage_erweitert.xlsx", index=False, encoding='utf8') 